# Elixir: protocols
* Protocols are a mechanism to achieve polymorphism in Elixir. Dispatching on a protocol is available to any data type as long as it implements the protocol.

* In Elixir, we have two idioms for the #items in a data structure: length and size. length means the information must be computed. For example, length(list) needs to traverse the whole list. tuple_size(tuple) and byte_size(binary) do not depend on the tuple and binary size as the size information is pre-computed in the data structure.

* Even if we have type-specific functions for getting the size built into Elixir (such as tuple_size/1), we could create a generic Size protocol.

In [1]:
defprotocol Size do
  @doc "Calculates the size (and not the length!) of a data structure"
  def size(data)
end

{:module, Size, <<70, 79, 82, 49, 0, 0, 18, 88, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 2, 103, 0, 0, 0, 50, 11, 69, 108, 105, 120, 105, 114, 46, 83, 105, 122, 101, 8, 95, 95, 105, 110, 102, 111, 95, 95, 7, 99, ...>>, {:__protocol__, 1}}

* The Size protocol expects a function called size that receives one argument (the data structure we want to know the size of) to be implemented. We can now implement this protocol for the data structures that would have a compliant implementation:

In [2]:
defimpl Size, for: BitString do
  def size(string), do: byte_size(string)
end

defimpl Size, for: Map do
  def size(map), do: map_size(map)
end

defimpl Size, for: Tuple do
  def size(tuple), do: tuple_size(tuple)
end

{:module, Size.Tuple, <<70, 79, 82, 49, 0, 0, 5, 212, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 194, 0, 0, 0, 20, 17, 69, 108, 105, 120, 105, 114, 46, 83, 105, 122, 101, 46, 84, 117, 112, 108, 101, 8, 95, 95, 105, 110, ...>>, {:__impl__, 1}}

* Let's see what happens:

In [3]:
Size.size("foo")

3

In [4]:
Size.size({:ok, "hello"})

2

In [5]:
Size.size(%{label: "some label"})

1

* Passing a data type that doesn’t implement the protocol raises an error:

In [6]:
Size.size([1, 2, 3])

Protocol.UndefinedError: 1

### Protocols and structs
* The power of Elixir’s extensibility comes when protocols and structs are used together.
* We previously learned that although structs are maps, they do not share protocol implementations with maps. For example, MapSets (sets based on maps) are implemented as structs. Let’s try to use the Size protocol with a MapSet.

In [6]:
Size.size(%{})

0

In [7]:
set = %MapSet{} = MapSet.new

#MapSet<[]>

In [8]:
Size.size(set)

Protocol.UndefinedError: 1

* Structs require their own protocol implementation. Since a MapSet has its size precomputed and accessible through MapSet.size, we can define a Size implementation for it.

In [8]:
defimpl Size, for: MapSet do
  def size(set), do: MapSet.size(set)
end

{:module, Size.MapSet, <<70, 79, 82, 49, 0, 0, 5, 200, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 185, 0, 0, 0, 19, 18, 69, 108, 105, 120, 105, 114, 46, 83, 105, 122, 101, 46, 77, 97, 112, 83, 101, 116, 8, 95, 95, 105, ...>>, {:__impl__, 1}}

* You can design your own semantics for the size of your struct. Not only that, you could use structs to build more robust data types, like queues, and implement all relevant protocols, such as Enumerable and possibly Size, for this data type.

In [9]:
defmodule User do
  defstruct [:name, :age]
end

defimpl Size, for: User do
  def size(_user), do: 2
end

{:module, Size.User, <<70, 79, 82, 49, 0, 0, 5, 164, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 181, 0, 0, 0, 19, 16, 69, 108, 105, 120, 105, 114, 46, 83, 105, 122, 101, 46, 85, 115, 101, 114, 8, 95, 95, 105, 110, 102, ...>>, {:__impl__, 1}}

### Any
* Manually implementing protocols for all types can quickly become tedious. 
* In such cases, Elixir provides two options: we can explicitly derive the protocol implementation for our types or automatically implement the protocol for all types. In both cases, we need to implement the protocol for Any.

#### Deriving

In [10]:
defimpl Size, for: Any do
  def size(_), do: 0
end

{:module, Size.Any, <<70, 79, 82, 49, 0, 0, 5, 152, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 179, 0, 0, 0, 19, 15, 69, 108, 105, 120, 105, 114, 46, 83, 105, 122, 101, 46, 65, 110, 121, 8, 95, 95, 105, 110, 102, 111, ...>>, {:__impl__, 1}}

* This isn't useful. For example, it makes no sense to say that the size of a PID or an Integer is 0.
* However, should we be fine with the implementation for Any, to use such implementation we would need to tell our struct to explicitly derive the Size protocol.

In [11]:
defmodule OtherUser do
  @derive [Size]
  defstruct [:name, :age]
end

{:module, OtherUser, <<70, 79, 82, 49, 0, 0, 5, 188, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 186, 0, 0, 0, 18, 16, 69, 108, 105, 120, 105, 114, 46, 79, 116, 104, 101, 114, 85, 115, 101, 114, 8, 95, 95, 105, 110, 102, ...>>, %OtherUser{age: nil, name: nil}}

#### Fallback to Any
* Another alternative to @derive is to tell the protocol to fallback to Any when an implementation cannot be found. This can be achieved by setting @fallback_to_any to true in the protocol definition.

In [12]:
defprotocol Size do
  @fallback_to_any true
  def size(data)
end

  nofile:1



{:module, Size, <<70, 79, 82, 49, 0, 0, 17, 104, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 2, 68, 0, 0, 0, 47, 11, 69, 108, 105, 120, 105, 114, 46, 83, 105, 122, 101, 8, 95, 95, 105, 110, 102, 111, 95, 95, 7, 99, ...>>, {:__protocol__, 1}}

* For most protocols, raising an error when a protocol is not implemented is the proper behaviour. That said, assuming we have implemented Any as in the previous section:

In [13]:
defimpl Size, for: Any do
  def size(_), do: 0
end

  nofile:1



{:module, Size.Any, <<70, 79, 82, 49, 0, 0, 5, 152, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 179, 0, 0, 0, 19, 15, 69, 108, 105, 120, 105, 114, 46, 83, 105, 122, 101, 46, 65, 110, 121, 8, 95, 95, 105, 110, 102, 111, ...>>, {:__impl__, 1}}

### Built-in protocols
* For example the Enum module provides many functions that work with any data structure that implements the Enumerable protocol.

In [14]:
Enum.map [1, 2, 3], fn(x) -> x * 2 end

[2, 4, 6]

In [15]:
Enum.reduce 1..3, 0, fn(x, acc) -> x+acc end

6

* Another useful example is the String.Chars protocol, which specifies how to convert a data structure with characters to a string. It’s exposed via the to_string function.

In [16]:
to_string :hello

"hello"

* Notice that string interpolation in Elixir calls the to_string function.

In [17]:
"age: #{25}"

"age: 25"

* This only works because numbers implement the String.Chars protocol. Passing a tuple, for example, will lead to an error.

In [18]:
tuple = {1,2,3}

{1, 2, 3}

In [19]:
"tuple: #{tuple}"

Protocol.UndefinedError: 1

* When there is a need to “print” a more complex data structure, one can use the inspect function, based on the Inspect protocol.
* The Inspect protocol is used to transform any data structure into a readable textual representation. This is what tools like IEx use to print results.

In [19]:
"tuple: #{inspect tuple}"

"tuple: {1, 2, 3}"

* By convention, whenever the inspected value starts with #, it is representing a data structure in non-valid Elixir syntax. This means the inspect protocol is not reversible as information may be lost along the way.

In [20]:
inspect &(&1+2)

"#Function<6.128620087/1 in :erl_eval.expr/5>"